In [260]:
pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [261]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [262]:
#packege imports
from sklearn import preprocessing, linear_model
import numpy as np

import sklearn


In [274]:
#Data reading
import pandas as pd

df = pd.read_csv("new_house_data.csv")
print(df.columns)
df.drop(["id"],axis=1)
df.head()

Index(['Unnamed: 0', 'id', 'Location', 'lat-lon', 'Area', 'Bed', 'Bath',
       'Price'],
      dtype='object')


,Unnamed: 0,id,Location,lat-lon,Area,Bed,Bath,Price
0,0,0,"Block H, Bashundhara R-A, Dhaka",0.263433,"1,600 sqft",3,3,23 Thousand
1,1,1,"Farmgate, Tejgaon, Dhaka",0.262859,900 sqft,2,2,23 Thousand
2,3,3,"Gulshan 1, Gulshan, Dhaka",0.263010,"2,200 sqft",3,4,78 Thousand
3,4,4,"Baridhara, Dhaka",0.263252,"2,200 sqft",3,3,77 Thousand
4,5,5,"Bashundhara R-A, Dhaka",0.263335,"3,000 sqft",4,5,52 Thousand


In [275]:
# #Data processing
# # Process the "Area" column
# df["Area"] = df["Area"].astype(str).map(lambda i: i.replace(" sqft", "").replace(",", ""))
# df["Area"] = df["Area"].map(lambda i: int(i))  # Convert to integer after cleaning

# # Process the "Price" column
# df["Price"] = df["Price"].astype(str).map(lambda i: i.replace(" Thousand", "000"))
# df["Price"] = df["Price"].map(lambda i: i.replace(" Lakh", "00000").replace(".", ""))
# df["Price"] = df["Price"].map(lambda i: i.replace(" Lakh", "00000").replace(".", ""))
# df["Price"] = df["Price"].map(lambda i: int(i))  # Convert to integer after cleaning

# df.head()
# print(df.columns)
# Data processing
# Process the "Area" column
df["Area"] = df["Area"].astype(str).map(lambda i: i.replace(" sqft", "").replace(",", ""))
df["Area"] = df["Area"].map(lambda i: int(i))  # Convert to integer after cleaning

def process_price(price):
    try:
        
        price = str(price)
        
        if "Thousand" in price and "." in price:
            # Replace decimal point and " Thousand"
            return float(price.replace(" Thousand", "00").replace(".", "")) 
        elif "Thousand" in price:
            # Replace " Thousand" without decimals
            return int(price.replace(" Thousand", "000"))
        if "Lakh" in price and "." in price:
            # Replace decimal point and " Thousand"
            return float(price.replace(" Lakh", "000").replace(".", "")) 
        elif "Lakh" in price:
            # Replace " Lakh" and remove any decimal points
            return int(price.replace(" Lakh", "00000"))
        else:
            # Try converting to an integer directly
            #print("11111")
            return int(price)
    except (ValueError, AttributeError, TypeError) as e:
       # print(f"Error processing price: {price}. Error: {e}")
        return None  # Return None for problematic entries


# Apply the function to the Price column
df["Price"] = df["Price"].map(process_price)

# Check rows where processing failed
print("Rows with invalid Price values:")
print(df[df["Price"].isnull()])

# Optionally drop rows with invalid/missing Price
df = df.dropna(subset=["Price"])  # Drop rows where Price is None



# Display the processed DataFrame
df.head()
print(df.columns)



Rows with invalid Price values:
Empty DataFrame
Columns: [Unnamed: 0, id, Location, lat-lon, Area, Bed, Bath, Price]
Index: []
Index(['Unnamed: 0', 'id', 'Location', 'lat-lon', 'Area', 'Bed', 'Bath',
       'Price'],
      dtype='object')


In [276]:
#df = df[df['Price'] >= 0]  # Remove rows with negative prices
print(df.describe())


       Unnamed: 0          id     lat-lon         Area         Bed  \
count  434.000000  434.000000  434.000000   434.000000  434.000000   
mean   250.919355  250.919355    0.263363  1685.064516    2.937788   
std    146.640738  146.640738    0.000447   799.152447    0.632675   
min      0.000000    0.000000    0.262325   400.000000    1.000000   
25%    120.500000  120.500000    0.263076  1200.000000    3.000000   
50%    256.500000  256.500000    0.263352  1500.000000    3.000000   
75%    376.750000  376.750000    0.263465  2100.000000    3.000000   
max    498.000000  498.000000    0.264216  5500.000000    6.000000   

             Bath          Price  
count  434.000000     434.000000  
mean     3.062212   54395.161290  
std      0.872098   59909.804798  
min      1.000000    6000.000000  
25%      3.000000   22000.000000  
50%      3.000000   32000.000000  
75%      4.000000   63000.000000  
max      6.000000  410000.000000  


In [277]:
#Checking null value
print(df.columns)

df.isnull().sum()

Index(['Unnamed: 0', 'id', 'Location', 'lat-lon', 'Area', 'Bed', 'Bath',
       'Price'],
      dtype='object')


Unnamed: 0    0
id            0
Location      0
lat-lon       0
Area          0
Bed           0
Bath          0
Price         0
dtype: int64

In [278]:
#Spliting Data
df = df.drop(["Location"],axis=1)

x = np.array(df.drop(["Price"],axis=1))
#print(df.describe())

y = np.array(df["Price"])
print("x",x.shape)
print("y",y.shape)

xTrain,xTest, yTrain, yTest = sklearn.model_selection.train_test_split(x,y,
                                                                      test_size = 0.3,
                                                                      random_state = 10)
print(xTrain.shape)
print(xTest.shape)


x (434, 6)
y (434,)
(303, 6)
(131, 6)


In [279]:
#model Training
# model = LinearRegression(positive=True)
model = linear_model.LinearRegression()
model.fit(xTrain,yTrain)
accuracy = model.score(xTest,yTest)

print("Coefficiants",model.coef_)
print("Y intercept",round(model.intercept_,3))

print("Accuracy",round(accuracy*100,3),"%")

Coefficiants [-7.77951745e+00 -7.77952063e+00 -5.52482665e+06  7.69607448e+01
 -2.29458383e+03 -1.82633151e+04]
Y intercept 1446770.133
Accuracy 71.129 %


In [280]:
#Evaluation
testvals = model.predict(xTest)
# print(testvals)

error = []

for i,testval in enumerate(testvals):
    error.append(yTest[i]-testval)
    
    print(f"Actual value {yTest[i]}; prediction {int(testval)}; Error: {round(error[i])}")
    

Actual value 64000.0; prediction 28431; Error: 35568
Actual value 68000.0; prediction 76393; Error: -8393
Actual value 11000.0; prediction 115317; Error: -104317
Actual value 245000.0; prediction 141741; Error: 103259
Actual value 32000.0; prediction -3141; Error: 35141
Actual value 12000.0; prediction 3168; Error: 8832
Actual value 115000.0; prediction 159186; Error: -44186
Actual value 22000.0; prediction 41479; Error: -19479
Actual value 42000.0; prediction 44952; Error: -2953
Actual value 100000.0; prediction 70146; Error: 29853
Actual value 40000.0; prediction 68926; Error: -28926
Actual value 27000.0; prediction 27793; Error: -794
Actual value 27000.0; prediction 33754; Error: -6755
Actual value 19000.0; prediction 35651; Error: -16651
Actual value 75000.0; prediction 124205; Error: -49205
Actual value 32000.0; prediction 32833; Error: -833
Actual value 35000.0; prediction 41725; Error: -6725
Actual value 26000.0; prediction 40041; Error: -14042
Actual value 16000.0; prediction 2